## Working with databases
- SQLite/ MySQL/ MariaDB/ PostgreSQL: SQL-based database technology

### Tasks
1. decide on a structure for data, then create database tables
2. add data values to database tables
3. extract data from database tables

### Task1-decide on a structure for data, then create database tables


In [1]:
# `DBcm` module
import DBcm

db_details = "CoachDB.sqlite3"

In [2]:
# `fetchall method` returns the output from the most-recently executed command as a list of tuples, which are assigned to variable `results`
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    results = db.fetchall()

In [3]:
# "pragma table_list": command sent to database
# returns any tables that are internal to SQLite
# -> "sqlite_schema", "sqlite_temp_schema"
results

[('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

In [4]:
SQL = """ 
    select time()
    """

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall() # `fetchall()`: list of tuples, `fetchone()`: single tuple
    
results

[('19:09:00',)]

In [5]:
SQL = """
    create table if not exists swimmers (
        id integer not null primary key autoincrement,
        name varchar(32) not null,
        age integer not null
    )
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)

In [6]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    results = db.fetchall()
results

[('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'swimmers', 'table', 3, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

In [7]:
with DBcm.UseDatabase(db_details) as db:
    SQL_1 = """
        create table if not exists events (
            id integer not null primary key autoincrement,
            distance varchar(16) not null,
            stroke varchar(16) not null
        )
    """
    SQL_2 = """
        create table if not exists times (
        swimmer_id integer not null,
        event_id integer not null,
        time varchar(16) not null,
        ts timestamp default current_timestamp
        )
    """
    db.execute(SQL_1)
    db.execute(SQL_2)

with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    results = db.fetchall()
results

[('main', 'times', 'table', 4, 0, 0),
 ('main', 'events', 'table', 3, 0, 0),
 ('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'swimmers', 'table', 3, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

### Task2-add data values to database tables

#### `swimmers` table

In [8]:
import os
import pyprojroot as root

FD = "data/swimdata/"

files = os.listdir(root.here(FD))
files.remove(".DS_Store")

In [9]:
first = files[0]
print(first)
name, age, _, _ = first.removesuffix(".txt").split("-")
name, age

Abi-10-100m-Back.txt


('Abi', '10')

In [10]:
import DBcm

db_details = "CoachDB.sqlite3"

In [11]:
SQL_INSERT = """ 
    insert into swimmers
    (name, age)
    values
    (?, ?)
"""

In [12]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_INSERT, (name, age))


In [13]:
SQL = """ 
    select * from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[(1, 'Abi', 10)]

In [14]:
SQL_DELETE = """ 
    delete from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE)

In [15]:
SQL = """ 
    select * from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[]

In [16]:
import os
import pyprojroot as here

FD = "data/swimdata/"
db_details = "CoachDB.sqlite3"

files = os.listdir(here.here(FD))
files.remove(".DS_Store")

SQL_SELECT = """
    select * from swimmers
    where name = ? and age = ?
"""

SQL_INSERT = """
    insert into swimmers
    (name, age)
    values
    (?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, _, _ = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT, (name, age))
        if db.fetchall():
            continue
        db.execute(SQL_INSERT, (name, age))

#### `events` table

In [18]:
import os
import pyprojroot as here

FD = "data/swimdata/"
db_details = "CoachDB.sqlite3"

files = os.listdir(here.here(FD))
files.remove(".DS_Store")

SQL_SELECT = """
    select * from events
    where distance = ? and stroke = ?
"""

SQL_INSERT = """
    insert into events
    (distance, stroke)
    values
    (?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        _, _, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT, (distance, stroke))
        if db.fetchall():
            continue
        db.execute(SQL_INSERT, (distance, stroke))

In [19]:
SQL = """ select * from events"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[(1, '100m', 'Back'),
 (2, '100m', 'Breast'),
 (3, '50m', 'Back'),
 (4, '50m', 'Breast'),
 (5, '50m', 'Free'),
 (6, '100m', 'Free'),
 (7, '200m', 'Back'),
 (8, '100m', 'Fly'),
 (9, '50m', 'Fly'),
 (10, '200m', 'IM'),
 (11, '200m', 'Breast'),
 (12, '200m', 'Free'),
 (13, '400m', 'Free')]

In [20]:
SQL = """ select * from swimmers"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[(2, 'Abi', 10),
 (3, 'Ali', 12),
 (4, 'Alison', 14),
 (5, 'Aurora', 13),
 (6, 'Bill', 18),
 (7, 'Blake', 15),
 (8, 'Calvin', 9),
 (9, 'Carl', 15),
 (10, 'Chris', 17),
 (11, 'Darius', 13),
 (12, 'Dave', 17),
 (13, 'Elba', 14),
 (14, 'Emma', 13),
 (15, 'Erika', 15),
 (16, 'Hannah', 13),
 (17, 'Katie', 9),
 (18, 'Lizzie', 14),
 (19, 'Maria', 9),
 (20, 'Mike', 15),
 (21, 'Owen', 15),
 (22, 'Ruth', 13),
 (23, 'Tasmin', 15)]

#### `times` table

In [21]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_info(times)")
    results = db.fetchall()

results

[(0, 'swimmer_id', 'INTEGER', 1, None, 0),
 (1, 'event_id', 'INTEGER', 1, None, 0),
 (2, 'time', 'varchar(16)', 1, None, 0),
 (3, 'ts', 'timestamp', 0, 'current_timestamp', 0)]

In [22]:
SQL_GET_SWIMMER = """ 
    select id from swimmers
    where name = ? and age = ?
"""

SQL_GET_EVENT = """
    select id from events
    where distance = ? and stroke = ?
"""

SQL_INSERT = """ 
    insert into times
    (swimmer_id, event_id, time)
    values
    (?, ?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_GET_SWIMMER, (name, age))
        s_id = db.fetchone()[0]
        db.execute(SQL_GET_EVENT, (distance, stroke))
        e_id = db.fetchone()[0]
        with open(here.here(FD + fn)) as sf:
            times = sf.read().strip().split(",")
            for t in times:
                db.execute(SQL_INSERT, (s_id, e_id, t))

In [23]:
# `count(*)`: count number of rows
SQL = """ select count(*) from times"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchone()[0]

results

338

In [24]:
# display the first 10 rows
SQL = """
    select * from times limit 10
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[(2, 1, '1:31.59', '2023-09-14 21:41:01'),
 (2, 1, '1:26.55', '2023-09-14 21:41:01'),
 (2, 1, '1:28.75', '2023-09-14 21:41:01'),
 (2, 1, '1:39.79', '2023-09-14 21:41:01'),
 (2, 1, '1:32.37', '2023-09-14 21:41:01'),
 (2, 2, '1:42.97', '2023-09-14 21:41:01'),
 (2, 2, '1:43.31', '2023-09-14 21:41:01'),
 (2, 2, '1:43.50', '2023-09-14 21:41:01'),
 (2, 2, '1:40.34', '2023-09-14 21:41:01'),
 (2, 3, '41.50', '2023-09-14 21:41:01')]